<a href="https://colab.research.google.com/github/agnaldoseixas/BootCamp_Analise_Dados/blob/main/apostila_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0 - Introdução

In [ ]:
'''
Definição: Ferramenta de processamento distribuido de dados que nos permite lidar
com grandes volumes de dados. Algo que o pandas ou o excel não conseguiriam ou
teriam de dificuldade.

Lazy Execution:
  - As operações são realizadas de forma preguiçosa;
  - Elas não são executadas imediatamente após serem chamadas, o Spark constroi
  um grafo de execução (também pode ser chamado de DAG ou de plano de execução) e
  só executa as operações quando fazemos uma chamada para ação.

Operações de Transformação:
  - Cria um plano lógico de execução;
  - Não são executadas imediatamente, mas preparam os dados para serem processados;
  - Exemplos: filter(), map(), select();

Operações de Ação:
  - Disparam a execução do plano lógico criado pelas operações de transformação.
  - Normalmente realiza algum cálculo/ Salva algo / Retorna um resultado
  - count(), show(), collect()

Operações de transformação são lazy e as de ação são eager.
'''

'\nDefinição: Ferramenta de processamento distribuido de dados que nos permite lidar\ncom grandes volumes de dados. Algo que o pandas ou o excel não conseguiriam ou\nteriam de dificuldade.\n\nLazy Execution:\n  - As operações são realizadas de forma preguiçosa;\n  - Elas não são executadas imediatamente após serem chamadas, o Spark constroi\n  um grafo de execução (também pode ser chamado de DAG ou de plano de execução) e\n  só executa as operações quando fazemos uma chamada para ação.\n\nOperações de Transformação:\n  - Cria um plano lógico de execução;\n  - Não são executadas imediatamente, mas preparam os dados para serem processados;\n  - Exemplos: filter(), map(), select();\n\nOperações de Ação:\n  - Disparam a execução do plano lógico criado pelas operações de transformação.\n  - Normalmente realiza algum cálculo/ Salva algo / Retorna um resultado\n  - count(), show(), collect()\n\nOperações de transformação são lazy e as de ação são eager.\n'

#1 - Preparando o ambiente

In [ ]:
!pip install pyspark

In [ ]:
'''
SparkSession.builder - Cria um construtor para configurar a nossa sessão.

appName('PySparkSQL') - Definindo o nome da nossa aplicação spark

getOrCreate() - Cria a sessão se ela não existir, caso já exista, retorna
a sessão existente

'''
#porta de entrada para interagir com os dados no pyspark
from pyspark.sql import SparkSession

#criando a sessão
spark = SparkSession.builder.appName('PySparkSQL').getOrCreate()

In [ ]:
print("Versão do Spark",spark.version)

Versão do Spark 3.5.1


In [ ]:
'''
Traz todas as funções que são usadas para manipulação de dados, indo desde a
transformação até a agregação.
'''
from pyspark.sql.functions import *
import pandas as pd

#2-Criando DataFrames

In [ ]:
# Criando um DataFrame em Pandas
nome = ['Nathan','Francisca','Matheus','Josa','Jean','Emily']
idade = [26,47,22,66,48,26]
altura = [1.73,1.60,1.80,1.62,1.73,1.63]
peso = [75,62,90,63,85,65]
sexo = ['M','F','M','F','M','F']

dfpd = pd.DataFrame({
    'nome': nome,
    'idade': idade,
    'altura': altura,
    'peso': peso,
    'sexo': sexo
})

display(dfpd)

,nome,idade,altura,peso,sexo
0,Nathan,26,1.73,75,M
1,Francisca,47,1.60,62,F
2,Matheus,22,1.80,90,M
3,Josa,66,1.62,63,F
4,Jean,48,1.73,85,M
5,Emily,26,1.63,65,F


In [ ]:
# Criando um DataFrame em pyspark
dados = [('Nathan',26,1.73,75,'M'),
         ('Francisca',47,1.60,62,'F'),
         ('Matheus',22,1.80,90,'M'),
         ('Josa',66,1.62,63,'F'),
         ('Jean',48,1.73,85,'M'),
         ('Emily',26,1.63,65,'F')]

columns = ['nome','idade','altura','peso','sexo']

dfps = spark.createDataFrame(dados, columns)
dfps.show()

+---------+-----+------+----+----+
|     nome|idade|altura|peso|sexo|
+---------+-----+------+----+----+
|   Nathan|   26|  1.73|  75|   M|
|Francisca|   47|   1.6|  62|   F|
|  Matheus|   22|   1.8|  90|   M|
|     Josa|   66|  1.62|  63|   F|
|     Jean|   48|  1.73|  85|   M|
|    Emily|   26|  1.63|  65|   F|
+---------+-----+------+----+----+



#3- Exibindo aleatoriamente uma linha do DF

In [ ]:
#pandas
dfpd.sample()

,nome,idade,altura,peso,sexo
3,Josa,66,1.62,63,F


In [ ]:
'''
withRaplacement: Determina como é feita a reposição da amostra
  False: Cada linha pode ser selecionada apenas 1 vez
  True: Uma linha pode ser selecionada várias vezes

Fração:
  Quais devem ser incluídos na amostra

Seed:
  Um número inteiro usado para controlar a aleatoriedade.
  Se for omitido, o pyspark usa um valor aleatório.
'''
#pyspark
dfps.sample(False,0.5).show(1)

+------+-----+------+----+----+
|  nome|idade|altura|peso|sexo|
+------+-----+------+----+----+
|Nathan|   26|  1.73|  75|   M|
+------+-----+------+----+----+
only showing top 1 row



#4- Exibindo linhas e colunas

In [ ]:
#pandas
dfpd.shape

(6, 5)

In [ ]:
#pyspark

num_linhas = dfps.count()
num_colunas = len(dfps.columns)

print(f'Número de linhas: {num_linhas}')
print(f'Número de colunas: {num_colunas}')

Número de linhas: 6
Número de colunas: 5


In [ ]:
#pyspark
print(f'Quantidade de linhas e colunas: {dfps.count(), len(dfps.columns)}')

Quantidade de linhas e colunas: (6, 5)


#5- Exibindo informações sobre os tipos de cada atributo

In [ ]:
#pandas
dfpd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   nome    6 non-null      object 
 1   idade   6 non-null      int64  
 2   altura  6 non-null      float64
 3   peso    6 non-null      int64  
 4   sexo    6 non-null      object 
dtypes: float64(1), int64(2), object(2)
memory usage: 372.0+ bytes


In [ ]:
#pandas
dfpd.dtypes

,0
nome,object
idade,int64
altura,float64
peso,int64
sexo,object


In [ ]:
'''
Integer - Inteiro até 32 bits
Bigint - Inteiro de até 64 bits
Float - Flutuante de até 32 bits
Double - Flutuante de até 64 bits
Decimal - Precisão Exata
String - String
'''
#spark
dfps.dtypes

[('nome', 'string'),
 ('idade', 'bigint'),
 ('altura', 'double'),
 ('peso', 'bigint'),
 ('sexo', 'string')]

In [ ]:
'''
Retorna o nome da coluna, o tipo e se ela aceita valores nulos
'''
#spark
dfps.printSchema()

root
 |-- nome: string (nullable = true)
 |-- idade: long (nullable = true)
 |-- altura: double (nullable = true)
 |-- peso: long (nullable = true)
 |-- sexo: string (nullable = true)



#6- Exibindo informações do indice das linhas

In [ ]:
#pandas
dfpd.index

RangeIndex(start=0, stop=6, step=1)

In [ ]:
'''
No pyspark os dataframe não possuem o conceito de índice (index) como nos
dataframes pandas. Isso ocorre porque o pyspark é projetado para trabalhar
com grandes volumes de dados distribuídos em um clusters, onde o conceito de
índice seria inviável.
'''

'\nNo pyspark os dataframe não possuem o conceito de índice (index) como nos\ndataframes pandas. Isso ocorre porque o pyspark é projetado para trabalhar\ncom grandes volumes de dados distribuídos em um clusters, onde o conceito de\níndice seria inviável.\n'

#7- Estatisticas Descritiva

In [ ]:
#pandas
medidas = dfpd.describe()
display(medidas)

,idade,altura,peso
count,6.000000,6.000000,6.000000
mean,39.166667,1.685000,73.333333
std,17.325319,0.079687,12.011106
min,22.000000,1.600000,62.000000
25%,26.000000,1.622500,63.500000
50%,36.500000,1.680000,70.000000
75%,47.750000,1.730000,82.500000
max,66.000000,1.800000,90.000000


In [ ]:
#pyspark
dfps.describe().show()

+-------+------+------------------+-------------------+------------------+----+
|summary|  nome|             idade|             altura|              peso|sexo|
+-------+------+------------------+-------------------+------------------+----+
|  count|     6|                 6|                  6|                 6|   6|
|   mean|  NULL|39.166666666666664| 1.6849999999999998| 73.33333333333333|NULL|
| stddev|  NULL|17.325318659888094|0.07968688725254612|12.011105971835677|NULL|
|    min| Emily|                22|                1.6|                62|   F|
|    max|Nathan|                66|                1.8|                90|   M|
+-------+------+------------------+-------------------+------------------+----+



#8- Exibindo as primeiras e ultimas linhas

In [ ]:
#pandas
dfpd.head()

,nome,idade,altura,peso,sexo
0,Nathan,26,1.73,75,M
1,Francisca,47,1.60,62,F
2,Matheus,22,1.80,90,M
3,Josa,66,1.62,63,F
4,Jean,48,1.73,85,M


In [ ]:
#pandas
dfpd.tail()

,nome,idade,altura,peso,sexo
1,Francisca,47,1.60,62,F
2,Matheus,22,1.80,90,M
3,Josa,66,1.62,63,F
4,Jean,48,1.73,85,M
5,Emily,26,1.63,65,F


In [ ]:
#primeiras linhas em spark
dfps.show(5)

+---------+-----+------+----+----+
|     nome|idade|altura|peso|sexo|
+---------+-----+------+----+----+
|   Nathan|   26|  1.73|  75|   M|
|Francisca|   47|   1.6|  62|   F|
|  Matheus|   22|   1.8|  90|   M|
|     Josa|   66|  1.62|  63|   F|
|     Jean|   48|  1.73|  85|   M|
+---------+-----+------+----+----+
only showing top 5 rows



In [ ]:
#ultimas linhas em spark
dfps.orderBy(col('nome').desc()).show(5)

+---------+-----+------+----+----+
|     nome|idade|altura|peso|sexo|
+---------+-----+------+----+----+
|   Nathan|   26|  1.73|  75|   M|
|  Matheus|   22|   1.8|  90|   M|
|     Josa|   66|  1.62|  63|   F|
|     Jean|   48|  1.73|  85|   M|
|Francisca|   47|   1.6|  62|   F|
+---------+-----+------+----+----+
only showing top 5 rows



#9- Exibindo Valores pelo Rótulo e Indice

In [ ]:
#Rotulo do Pandas
dfpd.loc[0]

,0
nome,Nathan
idade,26
altura,1.73
peso,75
sexo,M


In [ ]:
#Rotulo no pandas
medidas.loc['count']

,count
idade,6.0
altura,6.0
peso,6.0


In [ ]:
#indice no pandas
dfpd.iloc[0]

,0
nome,Nathan
idade,26
altura,1.73
peso,75
sexo,M


In [ ]:
#Rotulo no pyspak
dfps.where(col('nome') == 'Nathan').show()

+------+-----+------+----+----+
|  nome|idade|altura|peso|sexo|
+------+-----+------+----+----+
|Nathan|   26|  1.73|  75|   M|
+------+-----+------+----+----+



In [ ]:
#Rotulo no pyspak
dfps.where(col('altura') == 1.73).show()

+------+-----+------+----+----+
|  nome|idade|altura|peso|sexo|
+------+-----+------+----+----+
|Nathan|   26|  1.73|  75|   M|
|  Jean|   48|  1.73|  85|   M|
+------+-----+------+----+----+



In [ ]:
#Rotulo no pyspak
dfps.where(col('altura').between(1.73,1.90)).show()

+-------+-----+------+----+----+
|   nome|idade|altura|peso|sexo|
+-------+-----+------+----+----+
| Nathan|   26|  1.73|  75|   M|
|Matheus|   22|   1.8|  90|   M|
|   Jean|   48|  1.73|  85|   M|
+-------+-----+------+----+----+



In [ ]:
dfps.select('nome','idade').show()

+---------+-----+
|     nome|idade|
+---------+-----+
|   Nathan|   26|
|Francisca|   47|
|  Matheus|   22|
|     Josa|   66|
|     Jean|   48|
|    Emily|   26|
+---------+-----+



In [ ]:
#Índices no pyspark
'''
Em pyspark não temos suporte direto a indices, pois os dados são alocados em disco,
mas podemos fazer uma adaptação/ajuste técnico como veremos a seguir:
'''
dfps.select(dfps.columns[1]).show()

+-----+
|idade|
+-----+
|   26|
|   47|
|   22|
|   66|
|   48|
|   26|
+-----+



#13- Agrupamentos e Calculos Estatisticos

In [ ]:
#pandas
dfpd.groupby('sexo').size()

,0
sexo,
F,3
M,3


In [ ]:
#pandas
dfpd[['sexo','idade']].groupby('sexo').mean()

,idade
sexo,
F,46.333333
M,32.000000


In [ ]:
'''
dataframe.groupby('dimensão').agg(tipo_agregação('coluna_metrica').alias('apelido'))
'''
#pyspark
dfps.groupby("sexo").agg(round(avg("idade"),2).alias("idade_media")).show()

+----+-----------+
|sexo|idade_media|
+----+-----------+
|   F|      46.33|
|   M|       32.0|
+----+-----------+



In [ ]:
#pyspark
dfps.groupby("sexo").agg(round(max("idade"),2).alias("maior_idade")).show()

+----+-----------+
|sexo|maior_idade|
+----+-----------+
|   F|         66|
|   M|         48|
+----+-----------+



In [ ]:
#pyspark
dfps.groupby("sexo").agg(round(min("idade"),2).alias("media_menor")).show()

+----+-----------+
|sexo|media_menor|
+----+-----------+
|   F|         26|
|   M|         22|
+----+-----------+



In [ ]:
#pyspark
dfps.groupby("sexo").agg(round(sum("idade"),2).alias("soma_idade")).show()

+----+----------+
|sexo|soma_idade|
+----+----------+
|   F|       139|
|   M|        96|
+----+----------+



In [ ]:
dfps.groupby("sexo").agg(min("idade").alias("menor_idade"),
                         max("idade").alias("maior_idade"),
                         round(avg("idade"),2).alias("media_idade")).show()

+----+-----------+-----------+-----------+
|sexo|menor_idade|maior_idade|media_idade|
+----+-----------+-----------+-----------+
|   F|         26|         66|      46.33|
|   M|         22|         48|       32.0|
+----+-----------+-----------+-----------+



#11 - Pyspark x SQL

In [ ]:
!pip install mysql-connector-python

In [ ]:
import mysql.connector

In [ ]:
#parametros de conexão
host = '34.58.52.179'
user = 'root'
password = ''
database = 'loja'

In [ ]:
!curl ipecho.net/plain

35.204.158.67

In [ ]:
def consulta(query):
  try:
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    ) # Criando a conexão com o banco
    cursor = connection.cursor() # Criando um cursor que aponta para o banco
    cursor.execute(query) # Executando a query
    result = cursor.fetchall() # Armazenando os resultados da minha consulta
    tabelas = pd.DataFrame(result,columns = cursor.column_names) # Convertendo o resultado para um dataframe
    return tabelas

  except mysql.connector.Error as err:
    print(f'Erro: {err}')
    return None

  finally:
    if cursor:
      cursor.close()
    if connection:
      connection.close()

In [ ]:
query = '''
SHOW TABLES
'''
resultado = consulta(query)
display(resultado)

,Tables_in_loja
0,categorias
1,clientes
2,exemplo
3,fornecedores
4,funcionarios
5,produtos
6,vendas


In [ ]:
query = '''
SELECT *
FROM categorias;
'''
categorias = consulta(query)
display(categorias)

,id_categoria,nome
0,1,Eletrônicos
1,2,Roupas
2,3,Alimentos
3,4,Móveis
4,5,Cosméticos


In [ ]:
query = '''
SELECT *
FROM clientes;
'''
clientes = consulta(query)
display(clientes)

,id_cliente,nome,telefone,cidade,email,genero,endereco,renda,data_nascimento
0,1,João Silva,(11) 98765-4321,São Paulo,joao@email.com,Masculino,"Rua A, 123",5000.0,1980-05-10
1,2,Maria Santos,(21) 98765-5432,Rio de Janeiro,maria@email.com,Feminino,"Av. B, 456",4000.0,1985-08-15
2,3,Carlos Oliveira,(31) 98765-6543,Belo Horizonte,carlos@email.com,Masculino,"Rua C, 789",4500.0,1982-03-25
3,4,Ana Souza,(51) 98765-7654,Porto Alegre,ana@email.com,Feminino,"Av. D, 987",3800.0,1990-12-08
4,5,Pedro Santos,(41) 98765-8765,Curitiba,pedro@email.com,Masculino,"Rua E, 654",6000.0,1975-09-14
5,6,Luciana Lima,(11) 98765-9876,São Paulo,luciana@email.com,Feminino,"Rua F, 321",4200.0,1987-06-30
6,7,Rafael Costa,(21) 98765-8765,Rio de Janeiro,rafael@email.com,Masculino,"Av. G, 654",5500.0,1983-04-18
7,8,Mariana Oliveira,(31) 98765-7654,Belo Horizonte,mariana@email.com,Feminino,"Rua H, 987",4800.0,1989-02-22
8,9,Gustavo Martins,(51) 98765-6543,Porto Alegre,gustavo@email.com,Masculino,"Av. I, 234",7000.0,1980-10-15
9,10,Camila Silva,(41) 98765-5432,Curitiba,camila@email.com,Feminino,"Rua J, 876",3200.0,1992-07-05


In [ ]:
query = '''
SELECT *
FROM fornecedores;
'''
fornecedores = consulta(query)
display(fornecedores)

,id_fornecedor,nome,cidade,endereco,cnpj
0,1,Fornecedor A,São Paulo,"Rua G, 234",1234
1,2,Fornecedor B,Rio de Janeiro,"Av. H, 567",9876
2,3,Fornecedor C,Belo Horizonte,"Rua I, 890",5678


In [ ]:
query = '''
SELECT *
FROM funcionarios;
'''
funcionarios = consulta(query)
display(funcionarios)

,id_funcionario,nome,data_nascimento,genero,estado_civil,dependentes,cargo,salario
0,1,Fernando Silva,1982-07-15,Masculino,Casado,2,Vendedor,3500.0
1,2,Aline Costa,1980-12-20,Feminino,Solteira,0,Gerente,6000.0
2,3,Marcos Oliveira,1975-03-10,Masculino,Divorciado,1,Caixa,2800.0
3,4,Patrícia Souza,1988-06-05,Feminino,Solteira,0,Vendedora,3200.0
4,5,Rafael Martins,1983-09-18,Masculino,Casado,3,Gerente,6500.0
5,6,Carla Lima,1990-04-25,Feminino,Solteira,0,Vendedora,3000.0
6,7,Rodrigo Santos,1977-11-30,Masculino,Divorciado,2,Caixa,2800.0
7,8,Mariana Ferreira,1985-02-15,Feminino,Casada,1,Gerente,6000.0
8,9,Lucas Costa,1989-08-10,Masculino,Solteiro,0,Vendedor,3400.0
9,10,Ana Oliveira,1984-03-20,Feminino,Casada,2,Caixa,3000.0


In [ ]:
query = '''
SELECT *
FROM produtos;
'''
produtos = consulta(query)
display(produtos)

,id_produto,nome,preco,id_categoria,id_fornecedor
0,1,Smartphone X,1500.0,1,1
1,2,Notebook Y,3000.0,1,2
2,3,Camiseta Z,50.0,2,3
3,4,Arroz Integral,10.0,3,3
4,5,Sofá Confort,2500.0,4,1
5,6,Shampoo Revitalizante,20.0,5,2
6,7,"Monitor LED 24""",800.0,1,1
7,8,Calça Jeans,120.0,2,3
8,9,Feijão Carioca,8.0,3,3
9,10,Mesa de Jantar,1500.0,4,1


In [ ]:
query = '''
SELECT *
FROM vendas;
'''
vendas = consulta(query)
display(vendas)

,id_venda,id_produto,quantidade,id_cliente,id_funcionario,data
0,1,1,2,1,1,2024-07-01
1,2,3,1,2,2,2024-07-02
2,3,5,1,3,1,2024-07-03
3,4,2,1,4,3,2024-07-04
4,5,4,3,5,2,2024-07-05
...,...,...,...,...,...,...
118,119,18,1,39,7,2024-10-27
119,120,20,2,40,8,2024-10-28
120,121,1,1,1,9,2024-10-29
121,122,3,2,2,10,2024-10-30


#Convertendo as tabelas para um dataframe pyspark

In [ ]:
# Dataframe Pyspark
categorias = spark.createDataFrame(categorias.to_dict('records'))
categorias.show()

+------------+-----------+
|id_categoria|       nome|
+------------+-----------+
|           1|Eletrônicos|
|           2|     Roupas|
|           3|  Alimentos|
|           4|     Móveis|
|           5| Cosméticos|
+------------+-----------+



In [ ]:
'''
Criando uma view temporaria.
É necessário para que as consultas SQL sejam feitas.
'''
categorias.createOrReplaceTempView('categoriasv')

In [ ]:
# Dataframe Pyspark
clientes = spark.createDataFrame(clientes.to_dict('records'))
clientes.show()

+--------------+---------------+------------------+-----------+---------+----------+-----------------+------+---------------+
|        cidade|data_nascimento|             email|   endereco|   genero|id_cliente|             nome| renda|       telefone|
+--------------+---------------+------------------+-----------+---------+----------+-----------------+------+---------------+
|     São Paulo|     1980-05-10|    joao@email.com| Rua A, 123|Masculino|         1|       João Silva|5000.0|(11) 98765-4321|
|Rio de Janeiro|     1985-08-15|   maria@email.com| Av. B, 456| Feminino|         2|     Maria Santos|4000.0|(21) 98765-5432|
|Belo Horizonte|     1982-03-25|  carlos@email.com| Rua C, 789|Masculino|         3|  Carlos Oliveira|4500.0|(31) 98765-6543|
|  Porto Alegre|     1990-12-08|     ana@email.com| Av. D, 987| Feminino|         4|        Ana Souza|3800.0|(51) 98765-7654|
|      Curitiba|     1975-09-14|   pedro@email.com| Rua E, 654|Masculino|         5|     Pedro Santos|6000.0|(41) 9876

In [ ]:
'''
Criando uma view temporaria.
É necessário para que as consultas SQL sejam feitas.
'''
clientes.createOrReplaceTempView('clientesv')

In [ ]:
# Dataframe Pyspark
fornecedores = spark.createDataFrame(fornecedores.to_dict('records'))
fornecedores.show()

+--------------+----+----------+-------------+------------+
|        cidade|cnpj|  endereco|id_fornecedor|        nome|
+--------------+----+----------+-------------+------------+
|     São Paulo|1234|Rua G, 234|            1|Fornecedor A|
|Rio de Janeiro|9876|Av. H, 567|            2|Fornecedor B|
|Belo Horizonte|5678|Rua I, 890|            3|Fornecedor C|
+--------------+----+----------+-------------+------------+



In [ ]:
'''
Criando uma view temporaria.
É necessário para que as consultas SQL sejam feitas.
'''
fornecedores.createOrReplaceTempView('fornecedoresv')

In [ ]:
# Dataframe Pyspark
funcionarios = spark.createDataFrame(funcionarios.to_dict('records'))
funcionarios.show()

+---------+---------------+-----------+------------+---------+--------------+----------------+-------+
|    cargo|data_nascimento|dependentes|estado_civil|   genero|id_funcionario|            nome|salario|
+---------+---------------+-----------+------------+---------+--------------+----------------+-------+
| Vendedor|     1982-07-15|          2|      Casado|Masculino|             1|  Fernando Silva| 3500.0|
|  Gerente|     1980-12-20|          0|    Solteira| Feminino|             2|     Aline Costa| 6000.0|
|    Caixa|     1975-03-10|          1|  Divorciado|Masculino|             3| Marcos Oliveira| 2800.0|
|Vendedora|     1988-06-05|          0|    Solteira| Feminino|             4|  Patrícia Souza| 3200.0|
|  Gerente|     1983-09-18|          3|      Casado|Masculino|             5|  Rafael Martins| 6500.0|
|Vendedora|     1990-04-25|          0|    Solteira| Feminino|             6|      Carla Lima| 3000.0|
|    Caixa|     1977-11-30|          2|  Divorciado|Masculino|           

In [ ]:
'''
Criando uma view temporaria.
É necessário para que as consultas SQL sejam feitas.
'''
funcionarios.createOrReplaceTempView('funcionariosv')

In [ ]:
# Dataframe Pyspark
produtos = spark.createDataFrame(produtos.to_dict('records'))
produtos.show()

+------------+-------------+----------+--------------------+------+
|id_categoria|id_fornecedor|id_produto|                nome| preco|
+------------+-------------+----------+--------------------+------+
|           1|            1|         1|        Smartphone X|1500.0|
|           1|            2|         2|          Notebook Y|3000.0|
|           2|            3|         3|          Camiseta Z|  50.0|
|           3|            3|         4|      Arroz Integral|  10.0|
|           4|            1|         5|        Sofá Confort|2500.0|
|           5|            2|         6|Shampoo Revitaliz...|  20.0|
|           1|            1|         7|     Monitor LED 24"| 800.0|
|           2|            3|         8|         Calça Jeans| 120.0|
|           3|            3|         9|      Feijão Carioca|   8.0|
|           4|            1|        10|      Mesa de Jantar|1500.0|
|           5|            2|        11|Condicionador Hid...|  25.0|
|           1|            1|        12|Fone de O

In [ ]:
'''
Criando uma view temporaria.
É necessário para que as consultas SQL sejam feitas.
'''
produtos.createOrReplaceTempView('produtosv')

In [ ]:
# Dataframe Pyspark
vendas = spark.createDataFrame(vendas.to_dict('records'))
vendas.show()

+----------+----------+--------------+----------+--------+----------+
|      data|id_cliente|id_funcionario|id_produto|id_venda|quantidade|
+----------+----------+--------------+----------+--------+----------+
|2024-07-01|         1|             1|         1|       1|         2|
|2024-07-02|         2|             2|         3|       2|         1|
|2024-07-03|         3|             1|         5|       3|         1|
|2024-07-04|         4|             3|         2|       4|         1|
|2024-07-05|         5|             2|         4|       5|         3|
|2024-07-06|         6|             4|         6|       6|         2|
|2024-07-07|         7|             5|         8|       7|         1|
|2024-07-08|         8|             6|        10|       8|         1|
|2024-07-09|         9|             7|        12|       9|         3|
|2024-07-10|        10|             8|        14|      10|         2|
|2024-07-11|        11|             9|        16|      11|         1|
|2024-07-12|        

In [ ]:
'''
Criando uma view temporaria.
É necessário para que as consultas SQL sejam feitas.
'''
vendas.createOrReplaceTempView('vendasv')

#Select

In [ ]:
'''
Consultando a view temporária
'''
select = spark.sql("SELECT nome, cidade FROM clientesv")
select.show()

+-----------------+--------------+
|             nome|        cidade|
+-----------------+--------------+
|       João Silva|     São Paulo|
|     Maria Santos|Rio de Janeiro|
|  Carlos Oliveira|Belo Horizonte|
|        Ana Souza|  Porto Alegre|
|     Pedro Santos|      Curitiba|
|     Luciana Lima|     São Paulo|
|     Rafael Costa|Rio de Janeiro|
| Mariana Oliveira|Belo Horizonte|
|  Gustavo Martins|  Porto Alegre|
|     Camila Silva|      Curitiba|
|    Felipe Santos|     São Paulo|
|       Carla Lima|Rio de Janeiro|
|   Bruno Oliveira|Belo Horizonte|
|   Patrícia Costa|  Porto Alegre|
|  Rodrigo Martins|      Curitiba|
| Juliana Ferreira|     São Paulo|
|    Gabriel Silva|Rio de Janeiro|
|    Vanessa Souza|Belo Horizonte|
|Fernando Oliveira|  Porto Alegre|
|   Larissa Santos|      Curitiba|
+-----------------+--------------+
only showing top 20 rows



In [ ]:
'''
Equivalente em pyspark
'''
select2 = clientes.select("nome","cidade")
select2.show()

+-----------------+--------------+
|             nome|        cidade|
+-----------------+--------------+
|       João Silva|     São Paulo|
|     Maria Santos|Rio de Janeiro|
|  Carlos Oliveira|Belo Horizonte|
|        Ana Souza|  Porto Alegre|
|     Pedro Santos|      Curitiba|
|     Luciana Lima|     São Paulo|
|     Rafael Costa|Rio de Janeiro|
| Mariana Oliveira|Belo Horizonte|
|  Gustavo Martins|  Porto Alegre|
|     Camila Silva|      Curitiba|
|    Felipe Santos|     São Paulo|
|       Carla Lima|Rio de Janeiro|
|   Bruno Oliveira|Belo Horizonte|
|   Patrícia Costa|  Porto Alegre|
|  Rodrigo Martins|      Curitiba|
| Juliana Ferreira|     São Paulo|
|    Gabriel Silva|Rio de Janeiro|
|    Vanessa Souza|Belo Horizonte|
|Fernando Oliveira|  Porto Alegre|
|   Larissa Santos|      Curitiba|
+-----------------+--------------+
only showing top 20 rows



#WHERE

In [ ]:
'''
Na view temporaria
'''
where = spark.sql('''SELECT *
                      FROM clientesv
                      WHERE cidade = "Curitiba"''')
where.show()

+--------+---------------+-----------------+-----------+---------+----------+---------------+------+---------------+
|  cidade|data_nascimento|            email|   endereco|   genero|id_cliente|           nome| renda|       telefone|
+--------+---------------+-----------------+-----------+---------+----------+---------------+------+---------------+
|Curitiba|     1975-09-14|  pedro@email.com| Rua E, 654|Masculino|         5|   Pedro Santos|6000.0|(41) 98765-8765|
|Curitiba|     1992-07-05| camila@email.com| Rua J, 876| Feminino|        10|   Camila Silva|3200.0|(41) 98765-5432|
|Curitiba|     1978-06-23|rodrigo@email.com| Av. O, 321|Masculino|        15|Rodrigo Martins|6500.0|(41) 98765-0987|
|Curitiba|     1993-08-25|larissa@email.com| Rua T, 432| Feminino|        20| Larissa Santos|3200.0|(41) 98765-5432|
|Curitiba|     1979-07-21| marcos@email.com| Av. Y, 987|Masculino|        25|  Marcos Santos|6500.0|(41) 98765-0987|
|Curitiba|     1994-09-10|  julia@email.com|Rua D1, 876| Feminin

In [ ]:
# Pyspark
where2 = clientes.filter(clientes.cidade=="Curitiba")
where2.show()

+--------+---------------+-----------------+-----------+---------+----------+---------------+------+---------------+
|  cidade|data_nascimento|            email|   endereco|   genero|id_cliente|           nome| renda|       telefone|
+--------+---------------+-----------------+-----------+---------+----------+---------------+------+---------------+
|Curitiba|     1975-09-14|  pedro@email.com| Rua E, 654|Masculino|         5|   Pedro Santos|6000.0|(41) 98765-8765|
|Curitiba|     1992-07-05| camila@email.com| Rua J, 876| Feminino|        10|   Camila Silva|3200.0|(41) 98765-5432|
|Curitiba|     1978-06-23|rodrigo@email.com| Av. O, 321|Masculino|        15|Rodrigo Martins|6500.0|(41) 98765-0987|
|Curitiba|     1993-08-25|larissa@email.com| Rua T, 432| Feminino|        20| Larissa Santos|3200.0|(41) 98765-5432|
|Curitiba|     1979-07-21| marcos@email.com| Av. Y, 987|Masculino|        25|  Marcos Santos|6500.0|(41) 98765-0987|
|Curitiba|     1994-09-10|  julia@email.com|Rua D1, 876| Feminin

# Distinct

In [ ]:
# View temporaria
distinct = spark.sql("SELECT DISTINCT cidade FROM clientesv")
distinct.show()

+--------------+
|        cidade|
+--------------+
|      Curitiba|
|Belo Horizonte|
|     São Paulo|
|Rio de Janeiro|
|  Porto Alegre|
+--------------+



In [ ]:
distinct2 = clientes.select("cidade").distinct().orderBy(col("cidade").asc())
distinct2.show()

+--------------+
|        cidade|
+--------------+
|Belo Horizonte|
|      Curitiba|
|  Porto Alegre|
|Rio de Janeiro|
|     São Paulo|
+--------------+



#LIMIT

In [ ]:
# view
limit = spark.sql(''' SELECT *
                      FROM clientesv
                      WHERE cidade="Belo Horizonte"
                      LIMIT 3 ''')
limit.show()

+--------------+---------------+-----------------+----------+---------+----------+----------------+------+---------------+
|        cidade|data_nascimento|            email|  endereco|   genero|id_cliente|            nome| renda|       telefone|
+--------------+---------------+-----------------+----------+---------+----------+----------------+------+---------------+
|Belo Horizonte|     1982-03-25| carlos@email.com|Rua C, 789|Masculino|         3| Carlos Oliveira|4500.0|(31) 98765-6543|
|Belo Horizonte|     1989-02-22|mariana@email.com|Rua H, 987| Feminino|         8|Mariana Oliveira|4800.0|(31) 98765-7654|
|Belo Horizonte|     1984-08-07|  bruno@email.com|Av. M, 432|Masculino|        13|  Bruno Oliveira|4500.0|(31) 98765-2109|
+--------------+---------------+-----------------+----------+---------+----------+----------------+------+---------------+



In [ ]:
#pyspark
limit2 = clientes.filter(clientes.cidade=='Belo Horizonte').limit(3)
limit2.show()

# limit2 = clientes.filter(clientes.cidade=='Belo Horizonte')
# limit2.show(3)

+--------------+---------------+-----------------+----------+---------+----------+----------------+------+---------------+
|        cidade|data_nascimento|            email|  endereco|   genero|id_cliente|            nome| renda|       telefone|
+--------------+---------------+-----------------+----------+---------+----------+----------------+------+---------------+
|Belo Horizonte|     1982-03-25| carlos@email.com|Rua C, 789|Masculino|         3| Carlos Oliveira|4500.0|(31) 98765-6543|
|Belo Horizonte|     1989-02-22|mariana@email.com|Rua H, 987| Feminino|         8|Mariana Oliveira|4800.0|(31) 98765-7654|
|Belo Horizonte|     1984-08-07|  bruno@email.com|Av. M, 432|Masculino|        13|  Bruno Oliveira|4500.0|(31) 98765-2109|
+--------------+---------------+-----------------+----------+---------+----------+----------------+------+---------------+



# Booleanos (E | OU | Não)

##AND

In [ ]:
and2 = spark.sql(''' SELECT *
                      FROM clientesv
                      WHERE cidade = "São Paulo"
                      AND genero="Masculino" ''')
and2.show()

+---------+---------------+-----------------+------------+---------+----------+-------------+------+---------------+
|   cidade|data_nascimento|            email|    endereco|   genero|id_cliente|         nome| renda|       telefone|
+---------+---------------+-----------------+------------+---------+----------+-------------+------+---------------+
|São Paulo|     1980-05-10|   joao@email.com|  Rua A, 123|Masculino|         1|   João Silva|5000.0|(11) 98765-4321|
|São Paulo|     1986-11-28| felipe@email.com|  Av. K, 567|Masculino|        11|Felipe Santos|5800.0|(11) 98765-4321|
|São Paulo|     1988-12-05|ricardo@email.com| Av. U, 1098|Masculino|        21| Ricardo Lima|5800.0|(11) 98765-4321|
|São Paulo|     1989-01-22|marcelo@email.com|Av. E1, 1098|Masculino|        31| Marcelo Lima|5800.0|(11) 98765-4321|
+---------+---------------+-----------------+------------+---------+----------+-------------+------+---------------+



In [ ]:
and3 = clientes.filter((clientes.cidade=='São Paulo') & (clientes.genero=='Masculino'))
and3.show()

+---------+---------------+-----------------+------------+---------+----------+-------------+------+---------------+
|   cidade|data_nascimento|            email|    endereco|   genero|id_cliente|         nome| renda|       telefone|
+---------+---------------+-----------------+------------+---------+----------+-------------+------+---------------+
|São Paulo|     1980-05-10|   joao@email.com|  Rua A, 123|Masculino|         1|   João Silva|5000.0|(11) 98765-4321|
|São Paulo|     1986-11-28| felipe@email.com|  Av. K, 567|Masculino|        11|Felipe Santos|5800.0|(11) 98765-4321|
|São Paulo|     1988-12-05|ricardo@email.com| Av. U, 1098|Masculino|        21| Ricardo Lima|5800.0|(11) 98765-4321|
|São Paulo|     1989-01-22|marcelo@email.com|Av. E1, 1098|Masculino|        31| Marcelo Lima|5800.0|(11) 98765-4321|
+---------+---------------+-----------------+------------+---------+----------+-------------+------+---------------+



##OR

In [ ]:
#spark sql
or1 = spark.sql(''' SELECT *
                      FROM clientesv
                      WHERE cidade = "São Paulo"
                      OR cidade = "Curitiba" ''')

or1.show()

+---------+---------------+-----------------+------------+---------+----------+----------------+------+---------------+
|   cidade|data_nascimento|            email|    endereco|   genero|id_cliente|            nome| renda|       telefone|
+---------+---------------+-----------------+------------+---------+----------+----------------+------+---------------+
|São Paulo|     1980-05-10|   joao@email.com|  Rua A, 123|Masculino|         1|      João Silva|5000.0|(11) 98765-4321|
| Curitiba|     1975-09-14|  pedro@email.com|  Rua E, 654|Masculino|         5|    Pedro Santos|6000.0|(41) 98765-8765|
|São Paulo|     1987-06-30|luciana@email.com|  Rua F, 321| Feminino|         6|    Luciana Lima|4200.0|(11) 98765-9876|
| Curitiba|     1992-07-05| camila@email.com|  Rua J, 876| Feminino|        10|    Camila Silva|3200.0|(41) 98765-5432|
|São Paulo|     1986-11-28| felipe@email.com|  Av. K, 567|Masculino|        11|   Felipe Santos|5800.0|(11) 98765-4321|
| Curitiba|     1978-06-23|rodrigo@email

In [ ]:
#Equivalente em Pyspark
# O | é o OR no Pyspark
# O & é o AND no Pyspark
or2 = clientes.filter((clientes.cidade=='São Paulo') | (clientes.cidade=='Curitiba'))
or2.show()

+---------+---------------+-----------------+------------+---------+----------+----------------+------+---------------+
|   cidade|data_nascimento|            email|    endereco|   genero|id_cliente|            nome| renda|       telefone|
+---------+---------------+-----------------+------------+---------+----------+----------------+------+---------------+
|São Paulo|     1980-05-10|   joao@email.com|  Rua A, 123|Masculino|         1|      João Silva|5000.0|(11) 98765-4321|
| Curitiba|     1975-09-14|  pedro@email.com|  Rua E, 654|Masculino|         5|    Pedro Santos|6000.0|(41) 98765-8765|
|São Paulo|     1987-06-30|luciana@email.com|  Rua F, 321| Feminino|         6|    Luciana Lima|4200.0|(11) 98765-9876|
| Curitiba|     1992-07-05| camila@email.com|  Rua J, 876| Feminino|        10|    Camila Silva|3200.0|(41) 98765-5432|
|São Paulo|     1986-11-28| felipe@email.com|  Av. K, 567|Masculino|        11|   Felipe Santos|5800.0|(11) 98765-4321|
| Curitiba|     1978-06-23|rodrigo@email

##NOT

In [ ]:
#View Spark
not1 = spark.sql(''' SELECT *
                      FROM clientesv
                      WHERE NOT cidade = "São Paulo" ''')
not1.show()

+--------------+---------------+------------------+-----------+---------+----------+-----------------+------+---------------+
|        cidade|data_nascimento|             email|   endereco|   genero|id_cliente|             nome| renda|       telefone|
+--------------+---------------+------------------+-----------+---------+----------+-----------------+------+---------------+
|Rio de Janeiro|     1985-08-15|   maria@email.com| Av. B, 456| Feminino|         2|     Maria Santos|4000.0|(21) 98765-5432|
|Belo Horizonte|     1982-03-25|  carlos@email.com| Rua C, 789|Masculino|         3|  Carlos Oliveira|4500.0|(31) 98765-6543|
|  Porto Alegre|     1990-12-08|     ana@email.com| Av. D, 987| Feminino|         4|        Ana Souza|3800.0|(51) 98765-7654|
|      Curitiba|     1975-09-14|   pedro@email.com| Rua E, 654|Masculino|         5|     Pedro Santos|6000.0|(41) 98765-8765|
|Rio de Janeiro|     1983-04-18|  rafael@email.com| Av. G, 654|Masculino|         7|     Rafael Costa|5500.0|(21) 9876

In [ ]:
# Equivalente em pyspark
# Usamos o != para o NOT
not2 = clientes.filter(col("cidade") != "São Paulo")
not2.show()

+--------------+---------------+------------------+-----------+---------+----------+-----------------+------+---------------+
|        cidade|data_nascimento|             email|   endereco|   genero|id_cliente|             nome| renda|       telefone|
+--------------+---------------+------------------+-----------+---------+----------+-----------------+------+---------------+
|Rio de Janeiro|     1985-08-15|   maria@email.com| Av. B, 456| Feminino|         2|     Maria Santos|4000.0|(21) 98765-5432|
|Belo Horizonte|     1982-03-25|  carlos@email.com| Rua C, 789|Masculino|         3|  Carlos Oliveira|4500.0|(31) 98765-6543|
|  Porto Alegre|     1990-12-08|     ana@email.com| Av. D, 987| Feminino|         4|        Ana Souza|3800.0|(51) 98765-7654|
|      Curitiba|     1975-09-14|   pedro@email.com| Rua E, 654|Masculino|         5|     Pedro Santos|6000.0|(41) 98765-8765|
|Rio de Janeiro|     1983-04-18|  rafael@email.com| Av. G, 654|Masculino|         7|     Rafael Costa|5500.0|(21) 9876

# Funções de Agregação

##SUM

In [ ]:
#view spark
sum1 = spark.sql(''' SELECT SUM(salario) AS soma_salario
                      FROM funcionariosv
                      WHERE genero ="Masculino" ''')
sum1.show()

+------------+
|soma_salario|
+------------+
|     19000.0|
+------------+



In [ ]:
# Equivalente em pyspark
# filter - filtro
# agg - para trabalhar com agregação
# alias - para dar um apelido na exibição
soma2 = funcionarios.filter(col("genero")=="Masculino").agg(sum("salario").alias("soma_salario"))
soma2.show()

+------------+
|soma_salario|
+------------+
|     19000.0|
+------------+



#AVG

In [ ]:
# View
avg1 = spark.sql(''' SELECT AVG(salario) AS media_salario
                      FROM funcionariosv
                      WHERE genero ="Feminino" ''')
avg1.show()

+-------------+
|media_salario|
+-------------+
|       4240.0|
+-------------+



In [ ]:
#pyspark
media2 = funcionarios.filter(col("genero")=="Feminino").agg(avg("salario").alias("media_salario"))
media2.show()

+-------------+
|media_salario|
+-------------+
|       4240.0|
+-------------+



## MAX

In [ ]:
#view
max1 = spark.sql(''' SELECT MAX(salario) AS maior_salario
                      FROM funcionariosv
                      WHERE genero ="Masculino" ''')
max1.show()

+-------------+
|maior_salario|
+-------------+
|       6500.0|
+-------------+



In [ ]:
#Pyspark
  max2 = funcionarios.filter(col("genero")=="Masculino").agg(max("salario").alias("maior_salario"))
max2.show()

+-------------+
|maior_salario|
+-------------+
|       6500.0|
+-------------+



## MIN

In [ ]:
# View
min1 = spark.sql(''' SELECT MIN(salario) AS menor_salario
                      FROM funcionariosv
                      WHERE genero ="Feminino" ''')
min1.show()

+-------------+
|menor_salario|
+-------------+
|       3000.0|
+-------------+



In [ ]:
#Pyspark
min2 = funcionarios.filter(col("genero")=="Feminino").agg(min("salario").alias("menor_salario"))
min2.show()

+-------------+
|menor_salario|
+-------------+
|       3000.0|
+-------------+



## COUNT

In [ ]:
#view
count1 = spark.sql(''' SELECT COUNT(id_funcionario) AS quantidade_funcionarios
                      FROM funcionariosv
                      WHERE genero ="Masculino" ''')
count1.show()

+-----------------------+
|quantidade_funcionarios|
+-----------------------+
|                      5|
+-----------------------+



In [ ]:
#Pyspark
count2 = funcionarios.filter(col("genero")=="Masculino").agg(count("id_funcionario").alias("quantidade_funcionarios"))
count2.show()

+-----------------------+
|quantidade_funcionarios|
+-----------------------+
|                      5|
+-----------------------+



In [ ]:
funcionarios.show()

+---------+---------------+-----------+------------+---------+--------------+----------------+-------+
|    cargo|data_nascimento|dependentes|estado_civil|   genero|id_funcionario|            nome|salario|
+---------+---------------+-----------+------------+---------+--------------+----------------+-------+
| Vendedor|     1982-07-15|          2|      Casado|Masculino|             1|  Fernando Silva| 3500.0|
|  Gerente|     1980-12-20|          0|    Solteira| Feminino|             2|     Aline Costa| 6000.0|
|    Caixa|     1975-03-10|          1|  Divorciado|Masculino|             3| Marcos Oliveira| 2800.0|
|Vendedora|     1988-06-05|          0|    Solteira| Feminino|             4|  Patrícia Souza| 3200.0|
|  Gerente|     1983-09-18|          3|      Casado|Masculino|             5|  Rafael Martins| 6500.0|
|Vendedora|     1990-04-25|          0|    Solteira| Feminino|             6|      Carla Lima| 3000.0|
|    Caixa|     1977-11-30|          2|  Divorciado|Masculino|           

#GROUP BY

In [ ]:
#View
group = spark.sql(''' SELECT genero, estado_civil, AVG(salario) AS media_salario, COUNT(salario) AS quantidade
                      FROM funcionariosv
                      GROUP BY genero, estado_civil ''')
group.show()

+---------+------------+------------------+----------+
|   genero|estado_civil|     media_salario|quantidade|
+---------+------------+------------------+----------+
| Feminino|    Solteira|4066.6666666666665|         3|
|Masculino|      Casado|            5000.0|         2|
|Masculino|  Divorciado|            2800.0|         2|
|Masculino|    Solteiro|            3400.0|         1|
| Feminino|      Casada|            4500.0|         2|
+---------+------------+------------------+----------+



In [ ]:
#Pyspark
group2 = funcionarios.groupBy("genero","estado_civil").agg(
            avg("salario").alias("media_salario"),
            count("salario").alias("quantidade"),
            min("salario").alias("menor_salario"),
            max("salario").alias("maior_salario")
          )
group2.show()

+---------+------------+------------------+----------+-------------+-------------+
|   genero|estado_civil|     media_salario|quantidade|menor_salario|maior_salario|
+---------+------------+------------------+----------+-------------+-------------+
| Feminino|    Solteira|4066.6666666666665|         3|       3000.0|       6000.0|
|Masculino|      Casado|            5000.0|         2|       3500.0|       6500.0|
|Masculino|  Divorciado|            2800.0|         2|       2800.0|       2800.0|
|Masculino|    Solteiro|            3400.0|         1|       3400.0|       3400.0|
| Feminino|      Casada|            4500.0|         2|       3000.0|       6000.0|
+---------+------------+------------------+----------+-------------+-------------+



# HAVING

In [ ]:
#View
having = spark.sql(''' SELECT genero, cargo, COUNT(salario) AS quantidade
                      FROM funcionariosv
                      GROUP BY genero, cargo, salario
                      HAVING salario > 3500 ''')
having.show()

+---------+-------+----------+
|   genero|  cargo|quantidade|
+---------+-------+----------+
|Masculino|Gerente|         1|
| Feminino|Gerente|         2|
+---------+-------+----------+



In [ ]:
#Pyspark
filtro = funcionarios.filter(col("salario")>3500)
having2 = filtro.groupBy("genero","cargo").agg(
          count("salario").alias("quantidade"))
having2.show()

+---------+-------+----------+
|   genero|  cargo|quantidade|
+---------+-------+----------+
| Feminino|Gerente|         2|
|Masculino|Gerente|         1|
+---------+-------+----------+



UNION

In [ ]:
union1 = spark.sql(''' SELECT nome, genero
                      FROM funcionariosv
                      WHERE genero = "Feminino"
                      UNION
                      SELECT nome, genero
                      FROM funcionariosv
                      WHERE genero = "Masculino" ''')
union1.show()

+----------------+---------+
|            nome|   genero|
+----------------+---------+
|     Aline Costa| Feminino|
|  Patrícia Souza| Feminino|
|Mariana Ferreira| Feminino|
|    Ana Oliveira| Feminino|
|      Carla Lima| Feminino|
|  Rafael Martins|Masculino|
| Marcos Oliveira|Masculino|
|  Fernando Silva|Masculino|
|  Rodrigo Santos|Masculino|
|     Lucas Costa|Masculino|
+----------------+---------+



In [ ]:
#Pyspark
feminino = funcionarios.filter(col("genero")=="Feminino").select("nome","genero")
masculino = funcionarios.filter(col("genero")=="Masculino").select("nome","genero")
union2 = feminino.union(masculino)
union2.show()

+----------------+---------+
|            nome|   genero|
+----------------+---------+
|     Aline Costa| Feminino|
|  Patrícia Souza| Feminino|
|      Carla Lima| Feminino|
|Mariana Ferreira| Feminino|
|    Ana Oliveira| Feminino|
|  Fernando Silva|Masculino|
| Marcos Oliveira|Masculino|
|  Rafael Martins|Masculino|
|  Rodrigo Santos|Masculino|
|     Lucas Costa|Masculino|
+----------------+---------+



# IN

In [ ]:
#View
in1 = spark.sql(''' SELECT id_produto, nome
                      FROM produtosv
                      WHERE id_produto IN (3,5,7,9) ''')
in1.show()

+----------+---------------+
|id_produto|           nome|
+----------+---------------+
|         3|     Camiseta Z|
|         5|   Sofá Confort|
|         7|Monitor LED 24"|
|         9| Feijão Carioca|
+----------+---------------+



In [ ]:
#Pyspark
in2 = produtos.filter(col("id_produto").isin([3,5,7,9])).select("id_produto","nome")
in2.show()

+----------+---------------+
|id_produto|           nome|
+----------+---------------+
|         3|     Camiseta Z|
|         5|   Sofá Confort|
|         7|Monitor LED 24"|
|         9| Feijão Carioca|
+----------+---------------+



# BETWEEN

In [ ]:
#View
between1 = spark.sql(''' SELECT id_produto, nome
                      FROM produtosv
                      WHERE id_produto BETWEEN 1 AND 7 ''')
between1.show()

+----------+--------------------+
|id_produto|                nome|
+----------+--------------------+
|         1|        Smartphone X|
|         2|          Notebook Y|
|         3|          Camiseta Z|
|         4|      Arroz Integral|
|         5|        Sofá Confort|
|         6|Shampoo Revitaliz...|
|         7|     Monitor LED 24"|
+----------+--------------------+



In [ ]:
#Pyspark
between2 = produtos.filter(col("id_produto").between(1,7)).select("id_produto","nome")
between2.show()

+----------+--------------------+
|id_produto|                nome|
+----------+--------------------+
|         1|        Smartphone X|
|         2|          Notebook Y|
|         3|          Camiseta Z|
|         4|      Arroz Integral|
|         5|        Sofá Confort|
|         6|Shampoo Revitaliz...|
|         7|     Monitor LED 24"|
+----------+--------------------+



# LIKE

## TERMINA COM

In [ ]:
#View
# % - sinaliza que tem uma sequencia de caracteres
like1 = spark.sql(''' SELECT nome
                      FROM funcionariosv
                      WHERE nome LIKE "%a" ''')
like1.show()

+----------------+
|            nome|
+----------------+
|  Fernando Silva|
|     Aline Costa|
| Marcos Oliveira|
|  Patrícia Souza|
|      Carla Lima|
|Mariana Ferreira|
|     Lucas Costa|
|    Ana Oliveira|
+----------------+



In [ ]:
#Pyspark
like2 = funcionarios.filter(col("nome").like("%a")).select("nome")
like2.show()

+----------------+
|            nome|
+----------------+
|  Fernando Silva|
|     Aline Costa|
| Marcos Oliveira|
|  Patrícia Souza|
|      Carla Lima|
|Mariana Ferreira|
|     Lucas Costa|
|    Ana Oliveira|
+----------------+



In [ ]:
#Pyspark
like2 = funcionarios.filter(col("nome").rlike("a$")).select("nome")
like2.show()

+----------------+
|            nome|
+----------------+
|  Fernando Silva|
|     Aline Costa|
| Marcos Oliveira|
|  Patrícia Souza|
|      Carla Lima|
|Mariana Ferreira|
|     Lucas Costa|
|    Ana Oliveira|
+----------------+



In [ ]:
exemplo = funcionarios.filter(col("nome").endswith("a")).select("nome")
exemplo.show()

+----------------+
|            nome|
+----------------+
|  Fernando Silva|
|     Aline Costa|
| Marcos Oliveira|
|  Patrícia Souza|
|      Carla Lima|
|Mariana Ferreira|
|     Lucas Costa|
|    Ana Oliveira|
+----------------+



## Começa Com

In [ ]:
#View
# % - sinaliza que tem uma sequencia de caracteres
like4 = spark.sql(''' SELECT nome
                      FROM funcionariosv
                      WHERE nome LIKE "A%" ''')
like4.show()

+------------+
|        nome|
+------------+
| Aline Costa|
|Ana Oliveira|
+------------+



In [ ]:
#Pyspark
like5 = funcionarios.filter(col("nome").like("A%")).select("nome")
like5.show()

+------------+
|        nome|
+------------+
| Aline Costa|
|Ana Oliveira|
+------------+



In [ ]:
#Pyspark
#Usamos o simbolo de ^ para ser o padrão no inicio de uma string
like6 = funcionarios.filter(col("nome").rlike("^A")).select("nome")
like6.show()

+------------+
|        nome|
+------------+
| Aline Costa|
|Ana Oliveira|
+------------+



In [ ]:
exemplo2 = funcionarios.filter(col("nome").startswith("A")).select("nome")
exemplo2.show()

+------------+
|        nome|
+------------+
| Aline Costa|
|Ana Oliveira|
+------------+



## Contém

In [ ]:
like7 = spark.sql(''' SELECT nome
                      FROM funcionariosv
                      WHERE nome LIKE "%na%" ''')
like7.show()

+----------------+
|            nome|
+----------------+
|  Fernando Silva|
|Mariana Ferreira|
|    Ana Oliveira|
+----------------+



In [ ]:
#Pyspark
like8 = funcionarios.filter(col("nome").like("%na%")).select("nome")
like8.show()

+----------------+
|            nome|
+----------------+
|  Fernando Silva|
|Mariana Ferreira|
|    Ana Oliveira|
+----------------+



In [ ]:
exemplo = funcionarios.filter(col("nome").contains("na")).select("nome")
exemplo.show()

+----------------+
|            nome|
+----------------+
|  Fernando Silva|
|Mariana Ferreira|
|    Ana Oliveira|
+----------------+



NULL

In [ ]:
#View
null1 = spark.sql(''' SELECT nome, cargo
                      FROM funcionariosv
                      WHERE cargo IS NULL ''')
null1.show()

+----+-----+
|nome|cargo|
+----+-----+
+----+-----+



In [ ]:
#Pyspark
null2 = funcionarios.filter(col("cargo").isNull()).select("nome","cargo")
null2.show()

+----+-----+
|nome|cargo|
+----+-----+
+----+-----+

